In [ ]:
!git clone https://github.com/idiap/coqui-ai-TTS

Cloning into 'coqui-ai-TTS'...
remote: Enumerating objects: 32686, done.
remote: Counting objects: 100% (106/106), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 32686 (delta 52), reused 37 (delta 31), pack-reused 32580 (from 2)
Receiving objects: 100% (32686/32686), 137.36 MiB | 14.79 MiB/s, done.
Resolving deltas: 100% (23620/23620), done.


In [ ]:
%cd coqui-ai-TTS
!pip install -e .

/content/coqui-ai-TTS
Obtaining file:///content/coqui-ai-TTS
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 97.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 115.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 22.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 13.3 MB/s eta 0:00:00


In [ ]:
%cd /content

/content


In [ ]:
!git clone https://huggingface.co/coqui/XTTS-v2

Cloning into 'XTTS-v2'...
remote: Enumerating objects: 161, done.
remote: Counting objects: 100% (161/161), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 161 (delta 69), reused 161 (delta 69), pack-reused 0 (from 0)
Receiving objects: 100% (161/161), 2.29 MiB | 7.60 MiB/s, done.
Resolving deltas: 100% (69/69), done.
Filtering content: 100% (4/4), 1.94 GiB | 47.30 MiB/s, done.


In [ ]:
%cd /content/coqui-ai-TTS

/content/coqui-ai-TTS


In [ ]:
from TTS.api import TTS
import time
tts = TTS(model_name="tts_models/multilingual/multi-dataset/xtts_v2", progress_bar=True, gpu=True)

/content/coqui-ai-TTS/TTS/api.py:91: UserWarning: `gpu` will be deprecated. Please use `tts.to(device)` instead.
  warnings.warn("`gpu` will be deprecated. Please use `tts.to(device)` instead.")


 > You must confirm the following:
 | > "I have purchased a commercial license from Coqui: licensing@coqui.ai"
 | > "Otherwise, I agree to the terms of the non-commercial CPML: https://coqui.ai/cpml" - [y/n]


100%|█████████▉| 1.86G/1.87G [00:21<00:00, 88.9MiB/s]
100%|██████████| 1.87G/1.87G [00:21<00:00, 86.0MiB/s]
100%|██████████| 4.37k/4.37k [00:00<00:00, 17.9kiB/s]
 80%|███████▉  | 288k/361k [00:00<00:00, 1.33MiB/s]
100%|██████████| 361k/361k [00:00<00:00, 633kiB/s] 
100%|██████████| 32.0/32.0 [00:00<00:00, 121iB/s]


In [ ]:
import torchaudio

In [ ]:
from IPython.display import Audio, display

In [ ]:
def format_text(text):
    text_list = text.split(".")
    final_texts = []
    i = 0

    while i < len(text_list):
        current = text_list[i].strip()

        if not current:  # boşsa atla
            i += 1
            continue

        # Kısa ise bir sonrakini birleştir
        if len(current) < 100 and i + 1 < len(text_list):
            next_part = text_list[i + 1].strip()
            if next_part:  # sonraki de boş değilse birleştir
                merged = current + ". " + next_part + "."
                final_texts.append(merged)
                i += 2
            else:
                final_texts.append(current + ".")
                i += 1
        else:
            final_texts.append(current + ".")
            i += 1

    return final_texts

In [ ]:
speaker_wav = "/content/oe_f5tts_2.wav"
text = "Selam, hoş geldin. Şimdi, getirdiğin parçaya bakınca hemen bir izlenim edindim. ve dürüst olmak gerekirse, oldukça etkileyici bir şey. Yüzeyde biraz aşınma var, ama genel olarak iyi durumda. Şimdi, bunun orijinal olup olmadığını kesin olarak öğrenmem için birkaç test yapmam gerekecek. Eğer düşündüğüm gibi bir şeyse, bu parça koleksiyoncular arasında ciddi talep görebilir. Tabii, her şey değeriyle ilgili. ve sana adil bir teklif sunmak istiyorum. Eğer bekleyebilirsen, detaylıca inceleyip sonra oturup konuşuruz. Anlaştık mı?"

In [ ]:
text = ""
ref_wav = ""
def create_speech(text,ref_wav):
  formatted = format_text(text)
  for i,t in enumerate(formatted):
    output_path = f"/content/cloned_voice_{i}.wav"

    start = time.time()
    # Ses dosyasını oluştur
    tts.tts_to_file(text=t, speaker_wav=speaker_wav, language="tr", file_path=output_path)
    print(time.time() - start)
    # Ses dosyasını çal
    ### play(output_path)